In [1]:
import math
import numpy as np
import numpy.random as npr
from numpy import linalg
import torch
import torch.nn as nn
from torch.utils import data
from torch.utils.data import Dataset
import torch.optim as optim
import time
from useful_tools import isPointinPolygon,polyshow,Genpoint,Distance_pol,GenBC
from math import *
torch.cuda.set_device(1)

In [2]:
import torch.nn as nn
import torch
import torch.nn.functional as F
class ResNet(nn.Module):
    def __init__(self,dim, m,o):
        super(ResNet, self).__init__()
        self.Ix = torch.zeros([dim,m]).cuda()
        self.Ix[0,0] = 1
        self.Ix[1,1] = 1
        self.fc1 = nn.Linear(dim, m)
        self.fc2 = nn.Linear(m, m)
        
        self.fc3 = nn.Linear(m, m)
        self.fc4 = nn.Linear(m, m)

        self.fc5 = nn.Linear(m, m)
        self.fc6 = nn.Linear(m, m)
        self.outlayer = nn.Linear(m, o)

    def forward(self, x):
        s = x@self.Ix
        y = self.fc1(x)
        y = F.relu(y)**deg
        y = self.fc2(y)
        y = F.relu(y)**deg
        y = y+s
        
        s=y
        y = self.fc3(y)
        y = F.relu(y)**deg
        y = self.fc4(y)
        y = F.relu(y)**deg
        y = y+s
        
        s=y
        y = self.fc5(y)
        y = F.relu(y)**deg
        y = self.fc6(y)
        y = F.relu(y)**deg
        y = y+s
        output = self.outlayer(y)
        return output
deg = 2

In [3]:
torch.set_default_tensor_type('torch.DoubleTensor')

In [4]:
def sol_exact(x):
    N_norm_s = torch.sum(x**2,dim=1) 
    N_u = (torch.exp(N_norm_s/2)).reshape([x.size()[0],1])
    return N_u
def sol_exact_H1(x):
    N_norm_s = torch.sum(x**2,dim=1)
    u = sol_exact(x)
    v= torch.ones(u.shape).cuda()
    sol = torch.zeros(x.size()[0],dim_set+1).cuda()
    sol[:,0] = u.reshape([1,x.size()[0]])
    sol[:,1:dim_set+1] = torch.autograd.grad(u,x,grad_outputs=v,create_graph=True)[0]
    return sol

In [5]:
def right(x):
    sol = torch.zeros(x.size()[0],dim_set,dim_set).cuda()
    N_norm_s = torch.sum(x**2,dim=1)
    u = sol_exact(x)
    v= torch.ones(u.shape).cuda()
    du = torch.autograd.grad(u,x,grad_outputs=v,create_graph=True)[0]
    for i in range(dim_set):
        ux_tem = du[:,i].reshape([x.size()[0],1])
        uxx_tem = torch.autograd.grad(ux_tem,x,grad_outputs=v,create_graph=True)[0]
        sol[:,:,i]= uxx_tem
    return sol

In [6]:
def generate_uniform_points_in_sphere(d,R,N):
    points = torch.randn(N,d)
    scales = (R*torch.rand(N))**(1/d)
    notes = points/(torch.sqrt(torch.sum(points**2,dim=1))/scales).reshape(N,1)*torch.ones(N ,d)
    return notes

In [7]:
dim_set = 4
Batch_size = 50000
def DGM1():
    x = generate_uniform_points_in_sphere(dim_set,1,Batch_size).cuda()
    x.requires_grad = True
    u = model_u(x)
    du = model_p(x)
    v= torch.ones(u.shape).cuda()
    ux = torch.autograd.grad(u,x,grad_outputs=v,create_graph=True)[0]
    loss1 = torch.sum((ux-du)**2)/Batch_size
    return loss1

In [8]:
def DGM2():
    x =  generate_uniform_points_in_sphere(dim_set,1,Batch_size).cuda()
    x.requires_grad = True
    u = model_u(x)
    v= torch.ones(u.shape).cuda()
    du = model_p(x)
    uxx = torch.zeros(Batch_size,dim_set,dim_set).cuda()
    for i in range(dim_set):
        ux_tem = du[:,i].reshape([x.size()[0],1])
        uxx_tem = torch.autograd.grad(ux_tem,x,grad_outputs=v,create_graph=True)[0]
        uxx[:,:,i] = uxx_tem
    det1 = torch.det(right(x))
    det2 = torch.det(uxx)
    loss1 = torch.sum((det1-det2)**2)/Batch_size
    return loss1

In [9]:
model_u1 = ResNet(dim_set,20,1)
model_p = ResNet(dim_set,20,dim_set)

device=torch.device("cuda:1" )
model_u1.to(device)
model_p.to(device)

ResNet(
  (fc1): Linear(in_features=4, out_features=20, bias=True)
  (fc2): Linear(in_features=20, out_features=20, bias=True)
  (fc3): Linear(in_features=20, out_features=20, bias=True)
  (fc4): Linear(in_features=20, out_features=20, bias=True)
  (fc5): Linear(in_features=20, out_features=20, bias=True)
  (fc6): Linear(in_features=20, out_features=20, bias=True)
  (outlayer): Linear(in_features=20, out_features=4, bias=True)
)

In [10]:
def model_u(x):
    return (torch.sum(x**2,dim=1)-1).reshape([x.size()[0],1])*model_u1(x) + exp(1/2)

In [11]:
def loss_error():
    notes_error = generate_uniform_points_in_sphere(dim_set,1,10000).cuda()
    notes_error.requires_grad= True
    exact = sol_exact_H1(notes_error)
    u_L2 = sqrt(torch.sum((exact[:,0])**2)/10000)
    Du_L2 = sqrt(torch.sum(exact[:,1:dim_set+1]**2)/10000)
    predict = torch.zeros(10000,2*dim_set+2).cuda()
    predict[:,0]= model_u(notes_error).reshape([1,notes_error.size()[0]])
    predict[:,1:dim_set+1]= model_p(notes_error)
    value1 = sqrt(torch.sum((predict[:,0] - exact[:,0])**2)/10000)/u_L2
    value2 = sqrt(torch.sum((predict[:,1:dim_set+1] - exact[:,1:dim_set+1])**2)/10000)/Du_L2
    return value1,value2

In [12]:
traintime = 10000
error_save=np.zeros([2,traintime])
optimizer = optim.Adam([
                {'params': model_u1.parameters()},
                {'params': model_p.parameters()}
            ])
i = 0
time_start=time.time()
for i in range(traintime):
    optimizer.zero_grad()
    losses = DGM1()+DGM2()
    losses.backward()
    optimizer.step() 
    error1,error2 = loss_error()
    error_save[0,i]=float(error1)
    error_save[1,i]=float(error2)
    i = i +1
    #scheduler.step()
    if i%500==1:
        print("i= ",i)
        print("error =",error1)
        print("loss1 =",losses.detach())
        np.save("error_save_LDG_MA_4D_w20_l3.npy", error_save)
np.save("error_save_LDG_MA_4D_w20_l3.npy", error_save)
time_end=time.time()
print('time cost',time_end-time_start,'s')

i=  1
error = 0.21659370837440076
loss1 = tensor(69.1919, device='cuda:1')
i=  501
error = 0.20591223550479396
loss1 = tensor(69.0569, device='cuda:1')
i=  1001
error = 0.334465799062272
loss1 = tensor(53.2097, device='cuda:1')
i=  1501
error = 0.41935932069700116
loss1 = tensor(0.1414, device='cuda:1')
i=  2001
error = 0.41491945228870764
loss1 = tensor(0.0713, device='cuda:1')
i=  2501
error = 0.41273945070402335
loss1 = tensor(0.0444, device='cuda:1')
i=  3001
error = 0.4166118127244218
loss1 = tensor(0.0404, device='cuda:1')
i=  3501
error = 0.411059906190446
loss1 = tensor(0.0260, device='cuda:1')


KeyboardInterrupt: 

In [ ]:
#导入模块
import numpy as np
import matplotlib.pyplot as plt

#建立步长为0.01，即每隔0.01取一个点
step = 0.01
x = np.arange(-1,1,step)
y = np.arange(-1,1,step)

X,Y = np.meshgrid(x,y)
an = []
for i in range(X.shape[0]):
    for j in range(Y.shape[1]):
        an.append([x[i],x[j]])
Z1 = model_u(torch.tensor(an).cuda()).reshape([X.shape[0],Y.shape[0]]).cpu().detach().numpy()
plt.contourf(X,Y,Z1)
plt.colorbar(ticks=np.exp([0, 0.25, 0.5, 0.75, 1]))

In [ ]:
Z2 = sol_exact(torch.tensor(an).cuda()).reshape([X.shape[0],Y.shape[0]]).cpu().detach().numpy()
plt.contourf(X,Y,Z2)
plt.colorbar(ticks=np.exp([0, 0.25, 0.5, 0.75, 1]))

In [ ]:
a = plt.contourf(X,Y,(Z1-Z2)**2)
plt.colorbar(ticks=np.exp([0, 0.25, 0.5, 0.75, 1]))

In [ ]:
model_u(torch.tensor([[1.0,0.0]]).cuda())

In [ ]:
sol_exact(torch.tensor([[1.0,0.0]]).cuda())